In [1]:
import csv
from math import ceil,floor
from constraint import*
import csp

In [2]:
def calculateDistribution(totalPopulationA,totalPopulationB,totalPopulationC,dailyLimit,cr1range,cr2range,cr3range,cr4range,cr5range):
#assume number of centre used is same from start till end then equally distribute number of vaccine per day based on it
    daysneeded = ceil((totalPopulationA+totalPopulationB+totalPopulationC)/dailyLimit)
    varange = floor(totalPopulationA/(daysneeded))
    vbrange = floor(totalPopulationB/(daysneeded))
    vcrange = floor(totalPopulationC/(daysneeded))
    problem2 = Problem()
    """
    #the variable declaration below ensures that the range for va,vb,vc,finalva,finalvb,finalvc will not deviate more than 150 
    from the initial expected value of varange,vbrange,vcrange that is total number of vacine per type needed divide by daysneeded
    """
    problem2.addVariable("va",[*range(varange-150,varange+150,1)])
    problem2.addVariable("vb",[*range(vbrange-150,vbrange+150,1)])
    problem2.addVariable("vc",[*range(vcrange-150,vcrange+150,1)])
    problem2.addVariable("finalva",[*range(varange-150,varange+150,1)])
    problem2.addVariable("finalvb",[*range(vbrange-150,vbrange+150,1)])
    problem2.addVariable("finalvc",[*range(vcrange-150,vcrange+150,1)])
    """
    the constraint below ensures that all 3 vaccine types are fully distributed after calculation of
    average day distribution * number of days needed -1 + final day distribution
    """
    problem2.addConstraint(lambda va,finalva:va*(daysneeded-1)+finalva==totalPopulationA,("va","finalva"))
    problem2.addConstraint(lambda vb,finalvb:vb*(daysneeded-1)+finalvb==totalPopulationB,("vb","finalvb"))
    problem2.addConstraint(lambda vc,finalvc:vc*(daysneeded-1)+finalvc==totalPopulationC,("vc","finalvc"))
    """
    the constraint below ensures that the maximum difference between average and final day of every vaccine distribution 
    will not exceed the difference of 150, this will significantly reduced the amount of combinations generated and runtime required
    """
    problem2.addConstraint(lambda va,finalva:abs(va-finalva)<=150,("va","finalva"))
    problem2.addConstraint(lambda vb,finalvb:abs(vb-finalvb)<=150,("vb","finalvb"))
    problem2.addConstraint(lambda vc,finalvc:abs(vc-finalvc)<=150,("vc","finalvc"))
    """
    the constraint below ensures that total of average and final day distribution will not exceed the daily limit inposed
    """
    problem2.addConstraint(lambda va,vb,vc:va+vb+vc <=dailyLimit,["va","vb","vc"])
    problem2.addConstraint(lambda finalva,finalvb,finalvc:finalva+finalvb+finalvc <=dailyLimit,["finalva","finalvb","finalvc"])
    solutions2=problem2.getSolutions()
    """
    this part will calculate the maximum population needed to cover per day to achieve optimum distribution 
    and also the difference between average daily and final day distribution
    """
    for a in range(len(solutions2)):
        solutions2[a]['differ'] =  ( abs(solutions2[a]['finalva']-solutions2[a]['va']) + abs(solutions2[a]['finalvb']-solutions2[a]['vb']) + abs(solutions2[a]['finalvc']-solutions2[a]['vc']) )
        maxb=solutions2[a]['finalva']+solutions2[a]['finalvb']+solutions2[a]['finalvc']
        maxa=solutions2[a]['va']+solutions2[a]['vb']+solutions2[a]['vc']
        if(maxa>maxb):
            maxpop=maxa
        else:
            maxpop=maxb
        maxpop -= maxpop % -100
        solutions2[a]['maxpop']=maxpop
    #this will sort the vaccineList according to the ascending value of difference calculated
    vaccineList = sorted(solutions2,key=lambda x:x['differ'])
    vaccineDistribution=min(solutions2,key=lambda x:x['differ'])
    """
    this part will calculate whether which one have the highest population needed among average daily and final day distribution
    and round off to nearest 100 ceiling value
    """
    maxb=vaccineDistribution['finalva']+vaccineDistribution['finalvb']+vaccineDistribution['finalvc']
    maxa=vaccineDistribution['va']+vaccineDistribution['vb']+vaccineDistribution['vc']
    if(maxa>maxb):
        maxpop=maxa
    else:
        maxpop=maxb
    maxpop -= maxpop % -100
    """
    the part below calculates all possible combinations of vaccine centres needed to cover the daily distribution based on 
    the maxpop calculated 
    """
    problem = Problem()
    problem.addVariable("cr1",[*range(0,cr1range+1,1)])
    problem.addVariable("cr2",[*range(0,cr2range+1,1)])
    problem.addVariable("cr3",[*range(0,cr3range+1,1)])
    problem.addVariable("cr4",[*range(0,cr4range+1,1)])
    problem.addVariable("cr5",[*range(0,cr5range+1,1)])
    #the constraint for maximum vaccination limit
    problem.addConstraint(lambda cr1,cr2,cr3,cr4,cr5:cr1*200 + cr2*500 + cr3*1000 + cr4*2500 + cr5*4000 >= maxpop,("cr1","cr2","cr3","cr4","cr5"))
    solutions=problem.getSolutions()
    #this loop will generate the cost for all possible combinations generated using csp above
    for a in range(len(solutions)):
        solutions[a]['cost'] = solutions[a]['cr1']*100 + solutions[a]['cr2']*250 +solutions[a]['cr3']*500 + solutions[a]['cr4']*800 + solutions[a]['cr5']*1200
    #this will sort the centreList according to the ascending value of cost needed
    centreList = sorted(solutions,key=lambda x:x['cost'])
    totalCentre =min(solutions,key=lambda x:x['cost'])
    centres = list(totalCentre.values())
    #below is the display of vaccination details for each states
    centreNeeded = "Vaccination Centre Type Needed             : "
    for a in range(5):
        if(totalCentre['cr'+str(a+1)]!=0):
            centreNeeded +=  "Centre " + str(a+1) + " x " + str(totalCentre['cr'+str(a+1)] ) + " "
    print("Total Day Needed         : " + str(daysneeded) + " Days")
    print("Rental per day           : RM " + str(centreList[0]['cost']))
    print("Total Vaccine A per day  : " + str(vaccineList[0]['va']))
    print("Total Vaccine A last day : " + str(vaccineList[0]['finalva']))
    print("Total Vaccine B per day  : " + str(vaccineList[0]['vb']))
    print("Total Vaccine B last day : " + str(vaccineList[0]['finalvb']))
    print("Total Vaccine C per day  : " + str(vaccineList[0]['vc']))
    print("Total Vaccine C last day : " + str(vaccineList[0]['finalvc']))
    print("Maximum total vaccine distribution per day : " + str(maxpop))
    print(centreNeeded)
    print()
    return vaccineList, centreList
"""
#displayCalculations will display top combinations for both vaccination distribution and 
also cost of different vaccine centres
"""
def displayCalculations(text,vaccineList,centreList):
    print(text)
    print("Top vaccine distribution for the smallest difference between final day and average day distribution")
    print ("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('Vaccine A','Vaccine B','Vaccine C', 'Final VA','Final VB','Final VC','Total Difference'))
    length=len(vaccineList)
    if(length>5):
        length=5
    for x in range(length):
        print ("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format(vaccineList[x]['va'],vaccineList[x]['vb'],vaccineList[x]['vc'], vaccineList[x]['finalva'],vaccineList[x]['finalvb'],vaccineList[x]['finalvc'],vaccineList[x]['differ']))
    print()
    print("Top cheapest vaccine centre usage")
    length=len(centreList)
    if(length>5):
        length=5
    for x in range(length):
        centreNeeded = "Total cost of RM " + str(centreList[x]['cost']) + " :"
        for a in range(5):
            if(centreList[x]['cr'+str(a+1)]!=0):
                centreNeeded +=  "Centre " + str(a+1) + " x " + str(centreList[x]['cr'+str(a+1)] ) + " "
        print(centreNeeded)
    print()

In [3]:
#assume number of centre used is same from start till end then equally distribute number of vaccine per day based on it
print("ST1")
vaccineList1, centreList1 =calculateDistribution(15000,434890,115900,5000,20,15,10,21,5)
print("ST2")
vaccineList2, centreList2 =calculateDistribution(35234,378860,100450,10000,30,16,15,10,2)
print("ST3")
vaccineList3, centreList3 =calculateDistribution(22318,643320,223400,7500,22,15,11,12,3)
print("ST4")
vaccineList4, centreList4 =calculateDistribution(23893,859900,269300,8500,16,16,16,15,1)
print("ST5")
vaccineList5, centreList5 =calculateDistribution(19284,450500,221100,9500,19,10,20,15,1)


displayCalculations("Combinations For ST1",vaccineList1,centreList1)
displayCalculations("Combinations For ST2",vaccineList2,centreList2)
displayCalculations("Combinations For ST3",vaccineList3,centreList3)
displayCalculations("Combinations For ST4",vaccineList4,centreList4)
displayCalculations("Combinations For ST5",vaccineList5,centreList5)

ST1
Total Day Needed         : 114 Days
Rental per day           : RM 1600
Total Vaccine A per day  : 132
Total Vaccine A last day : 84
Total Vaccine B per day  : 3815
Total Vaccine B last day : 3795
Total Vaccine C per day  : 1017
Total Vaccine C last day : 979
Maximum total vaccine distribution per day : 5000
Vaccination Centre Type Needed             : Centre 4 x 2 

ST2
Total Day Needed         : 52 Days
Rental per day           : RM 3200
Total Vaccine A per day  : 678
Total Vaccine A last day : 656
Total Vaccine B per day  : 7286
Total Vaccine B last day : 7274
Total Vaccine C per day  : 1932
Total Vaccine C last day : 1918
Maximum total vaccine distribution per day : 9900
Vaccination Centre Type Needed             : Centre 4 x 1 Centre 5 x 2 

ST3
Total Day Needed         : 119 Days
Rental per day           : RM 2400
Total Vaccine A per day  : 188
Total Vaccine A last day : 134
Total Vaccine B per day  : 5406
Total Vaccine B last day : 5412
Total Vaccine C per day  : 1877
Total V

In [ ]:
#this part saves all data generated into the csv
keys1 = vaccineList1[0].keys()
keys2 = centreList1[0].keys()
with open('st1vac.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys1)
    dict_writer.writeheader()
    dict_writer.writerows(vaccineList1)
with open('st1centre.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys2)
    dict_writer.writeheader()
    dict_writer.writerows(centreList1)
with open('st2vac.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys1)
    dict_writer.writeheader()
    dict_writer.writerows(vaccineList2)
with open('st2centre.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys2)
    dict_writer.writeheader()
    dict_writer.writerows(centreList2)
with open('st3vac.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys1)
    dict_writer.writeheader()
    dict_writer.writerows(vaccineList3)
with open('st3centre.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys2)
    dict_writer.writeheader()
    dict_writer.writerows(centreList3)
with open('st4vac.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys1)
    dict_writer.writeheader()
    dict_writer.writerows(vaccineList4)
with open('st4centre.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys2)
    dict_writer.writeheader()
    dict_writer.writerows(centreList4)
with open('st5vac.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys1)
    dict_writer.writeheader()
    dict_writer.writerows(vaccineList5)
with open('st5centre.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys2)
    dict_writer.writeheader()
    dict_writer.writerows(centreList5)